<a href="https://colab.research.google.com/github/Abd-Fattah/Database-Management-and-Practice--IBT3205-/blob/main/Lab%234.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import pandas as pd
%load_ext sql

In [4]:
conn = sqlite3.connect('lab_session.db')
c = conn.cursor()

In [5]:
# Define SQL commands
commands = [
    """
    CREATE TABLE IF NOT EXISTS students (
        student_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        major TEXT
    );
    """,
    """
    INSERT INTO students (first_name, last_name, major)
    VALUES ('John', 'Doe', 'Computer Science'),
           ('Jane', 'Doe', 'Mathematics'),
           ('Mike', 'Jordan', 'Physics');
    """,
    """
    CREATE TABLE IF NOT EXISTS courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT,
        department_id INTEGER,
        instructor_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id),
        FOREIGN KEY (instructor_id) REFERENCES instructors(instructor_id)
    );
    """,
    """
    INSERT INTO courses (course_name, department_id, instructor_id)
    VALUES ('Database Systems', 1, 1),
           ('Data Structures', 1, 2),
           ('Linear Algebra', 2, 3),
           ('Quantum Mechanics', 3, 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        course_id INTEGER,
        FOREIGN KEY (student_id) REFERENCES students(student_id),
        FOREIGN KEY (course_id) REFERENCES courses(course_id)
    );
    """,
    """
    INSERT INTO enrollments (student_id, course_id)
    VALUES (1, 1),
           (1, 2),
           (2, 3),
           (3, 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS instructors (
        instructor_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        department_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id)
    );
    """,
    """
    INSERT INTO instructors (first_name, last_name, department_id)
    VALUES ('Robert', 'Smith', 1),
           ('Susan', 'Johnson', 2),
           ('Michael', 'Brown', 3),
           ('Elizabeth', 'Taylor', 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS departments (
        department_id INTEGER PRIMARY KEY,
        department_name TEXT
    );
    """,
    """
    INSERT INTO departments (department_name)
    VALUES ('Computer Science'),
           ('Mathematics'),
           ('Physics'),
           ('Chemistry');
    """
]

# Execute SQL commands
for command in commands:
    c.execute(command)

# Commit changes and close connection
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('lab_session.db')

inner_joing_querry = """
SELECT students.first_name AS student_name, courses.course_name
FROM students
INNER JOIN enrollments ON students.student_id = enrollments.student_id
INNER JOIN courses ON enrollments.course_id = courses.course_id;
"""

inner_join_df = pd.read_sql_query(inner_joing_querry, conn)
inner_join_df

In [ ]:
# Execute LEFT JOIN query and visualize the result
left_join_query = """
SELECT students.first_name AS student_name, courses.course_name
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id;
"""

left_join_df = pd.read_sql_query(left_join_query, conn)
left_join_df

In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect('lab_session.db')

# Execute FULL JOIN query (emulated) and visualize the result
full_join_query_1 = """
WITH RECURSIVE full_join AS (
    SELECT students.first_name AS student_name, courses.course_name
    FROM students
    LEFT JOIN enrollments ON students.student_id = enrollments.student_id
    LEFT JOIN courses ON enrollments.course_id = courses.course_id
    UNION ALL
    SELECT students.first_name AS student_name, courses.course_name
    FROM courses
    LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
    LEFT JOIN students ON enrollments.student_id = students.student_id
    WHERE students.student_id IS NULL
)
SELECT * FROM full_join;
"""

full_join_df_1 = pd.read_sql_query(full_join_query_1, conn)
full_join_df_1


In [ ]:
conn = sqlite3.connect('lab_session.db')

subquerry_example1 = """
SELECT first_name, last_name
FROM Students
WHERE student_id IN (
  SELECT student_id
  FROM enrollments
  WHERE course_id IN (
    SELECT course_id
    FROM courses
    WHERE course_name = 'Database Systems'
  )
);
"""

subquerry_example1_df = pd.read_sql_query(subquerry_example1, conn)
subquerry_example1_df

In [ ]:
conn = sqlite3.connect('lab_session.db')

subquerry_example2 = """
SELECT course_name
FROM courses
WHERE department_id = (
  SELECT department_id
  FROM departments
  WHERE department_name = 'Computer Science'
);
"""

subquerry_example2_df = pd.read_sql_query(subquerry_example2, conn)
subquerry_example2_df

In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect('lab_session.db')

# Execute Aggregate Function query and visualize the result
agg_function_example_1 = """
SELECT departments.department_name, COUNT(DISTINCT instructors.instructor_id) AS instructor_count
FROM departments
LEFT JOIN instructors ON departments.department_id = instructors.department_id
GROUP BY departments.department_name;
"""

agg_df_1 = pd.read_sql_query(agg_function_example_1, conn)
agg_df_1

In [ ]:
# Execute Aggregate Function query and visualize the result
agg_function_example_2 = """
SELECT courses.course_name, COUNT(DISTINCT enrollments.student_id) AS student_count
FROM courses
LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
GROUP BY courses.course_name
HAVING student_count > 0;
"""

agg_df_2 = pd.read_sql_query(agg_function_example_2, conn)
agg_df_2

In [ ]:
agg_function_task1 = """
SELECT departments.department_name, COUNT(DISTINCT courses.course_id) AS course_count
FROM courses
LEFT JOIN departments ON courses.department_id = departments.department_id
GROUP BY departments.department_name
HAVING course_count > 0;
"""

agg_df_task1 = pd.read_sql_query(agg_function_task1, conn)
agg_df_task1

In [ ]:
agg_function_task2 = """
SELECT departments.department_name, COUNT(DISTINCT instructors.instructor_id) AS instructors_count
FROM instructors
LEFT JOIN departments ON instructors.department_id = departments.department_id
GROUP BY departments.department_name
HAVING instructors_count > 0;
"""

agg_df_task2 = pd.read_sql_query(agg_function_task2, conn)
agg_df_task2

In [ ]:
agg_function_task3 = """
SELECT
    departments.department_name,
    AVG(student_count) AS avg_students_per_course
FROM (
    SELECT
        courses.course_name,
        courses.department_id,
        COUNT(DISTINCT enrollments.student_id) AS student_count
    FROM courses
    LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
    GROUP BY courses.course_name, courses.department_id
) AS course_enrollments
JOIN departments ON course_enrollments.department_id = departments.department_id
GROUP BY departments.department_name;
"""

agg_df_task3 = pd.read_sql_query(agg_function_task3, conn)
agg_df_task3